# Homework 3

DUE DEC 1st at 11:59 PM


## Problem 1

In this problem, you will implement a simple feed-forward neural network using PyTorch, a straight-forward and simple-to-pickup framework for quickly prototyping deep learning model. 

PyTorch provides 2 powerful things. First, a nice data structure called Tensor (basically a matrix, similar to Numpy ndarray). Tensor is optimized for matrix calculation and can be loaded to a GPU. Tensor is also implemented so that it's easy to calculate and pass back chains of gradients, which is extremely useful for backpropagation on neural network. Second, a nice inner mechanism called Autograd that nicely map variables involved a chain of calculations and efficiently calculate their gradients via the chain rule when needed. Read more here: https://towardsdatascience.com/pytorch-autograd-understanding-the-heart-of-pytorchs-magic-2686cd94ec95  

You will train and evaluate the model on a simple dataset for classifying seeds. The dataset has 7 features and 3 labels. Refer to the example from the discussion session if needed. 

**In this problem, you are given a good amount of freedom in tweaking your choices in designing the network. As long as it's still a feed-forward network, it's fine. Accuracy does not matter as long as it's reasonable (like above 80%) but do try to achieve a high accuracy.**

(Optional): the dataset provided to you is very simple. Instead, you can challenge yourself by trying more difficult datasets. Some datasets that you can loaded directly using PyTorch:
- MNIST (Most famous dataset for getting into Deep Learning)
- Fashion-MNIST
- Kuzushiji-MNIST 

The datasets are available at https://pytorch.org/docs/stable/torchvision/datasets.html. You will get the same grade no matter what dataset you use. 

### Part a
Firstly, load and inspect the dataset from "seeds_dataset.csv". Split them into a train set (90%) and a test set (10%). You should be quite comfortable with these operations by now. 

### Part b 
Create a Python class for our neural network model. The network should have 1 input layer, at least 1 hidden layer, and 1 output layer. You are free to choose the size and the number of hidden layers (it may affect the performance so try tweaking around a bit), and the activation function (or no activation at all).

Some popular activation functions that you can try:
- Sigmoid (torch.sigmoid)
- ReLU (torch.relu)
- Tanh (torch.tanh)

In [1]:
import torch

class network(torch.nn.Module):
    
    # Create the network class by filling in this block of code

SyntaxError: ignored

### Part c 
Train the network using the train dataset. You are free to choose any suitable optimizer and loss function provided by PyTorch (or you can just use SGD optimizer and CrossEntropyLoss like we did in the discussion session for simplicity). After each epoch, record the current loss and the current accuracy. The current accuracy is obtained by evaluating the model on the train dataset. 

Some optimizers that you can try:
- SGD
- Adagrad
- Adam

https://pytorch.org/docs/stable/optim.html

In [ ]:
LEARNING_RATE = 0.01
EPOCHS = 20

for epoch in range(EPOCHS):
    
    # Train the network by filling in this block of code

Plot how the loss and the accuracy change over the epochs. 

### Part d 
Evaluate the model on the test dataset. Print out the accuracy. Does this accuracy agrees with the training accuracy showed on the plot? Why may they be different?

## Problem 2

If you haven't done so already, please read Chapter 2 (on fairness) of the textbook.

### Part a

Import the 'semi_synthetic.csv' dataset. This dataset is a modified version of the UCI Credit card dataset (details [here](https://archive.ics.uci.edu/ml/datasets/default+of+credit+card+clients)). We added a new synthetic feature to this dataset called `LIMIT_BAL`. It is highly predictive for `SEX==2` but not at all for `SEX==1`. As it turns out, this causes issues with fairness where models rely too much on `LIMIT_BAL` and discriminate against `SEX==2`. We will try to mitigate this in terms of the fairness criterion 'Statistical Parity' as it's mentioned in the textbook.

In [2]:
import pandas as pd

data = pd.read_csv("semi_synthetic.csv")

Extract features from the dataset (all columns except last) and call it `X`. Extract last column and call it `Y`. Also extract the `SEX` column and call this `SF` (for sensitive feature). Split all of these arrays data into train and test sets. You can use `sklearn.model_selection.train_test_split` and 20% ratio for the test dataset. Pass `stratify=Y` to `train_test_split` if you're using it. This ensures that both test and train datasets have the same ratio of 0/1 labels.

In [3]:
from sklearn.model_selection import train_test_split 
import numpy as np
import random
random.seed(2020)
np.random.seed(2020)

X = data.iloc[:,0:-1]
X = X.to_numpy()
Y = data.iloc[:,-1]
Y = Y.to_numpy()
SF = data.iloc[:,1]
SF = SF.to_numpy()
X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size = 0.20,stratify=Y)
SF_test = X_test[:,1]

Use a random forest classifier (whichever hyperparameters you want, you may leave it default) and train it using the train dataset (only `X_train` and `Y_train`. We will not use `SF_train` yet.)

In [4]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(max_depth=5, n_estimators=5, random_state=0)
clf.fit(X_train, y_train)
np.size(y_train)

24000

### Part b

Now we will consider the 'statistical parity' fairness metric on the test dataset. This basically means we compare the selection rates (i.e. predicted labels being 1) conditioned on sex and look at their difference/ratio. To do this, print 5 things: 

1. Overall selection rate (what percent is predicted as 1)
2. Selection rate for `SEX==1`
3. Selection rate for `SEX==2`
4. Difference between 2. and 3.
5. Ratio of 2. and 3.

Note that we didn't use anything related to how accurate the model is.

In [5]:
y_predict = clf.predict(X_test)
no1 = np.count_nonzero(y_predict)
overall_selection_rate = no1/np.size(y_predict)
print("The overall selection rate is ",overall_selection_rate)

sex1_count = 0
for i in range(np.size(y_predict)):
  if y_predict[i]==1 and X_test[i,1]==1:
    sex1_count +=1
print("Selection rate for SEX==1 is", sex1_count/np.size(y_predict))
sex2_count = no1-sex1_count
print("Selection rate for SEX==2 is", sex2_count/np.size(y_predict))
print("Difference of counts number between 2. and 3. is", sex2_count-sex1_count)
print("Ratio of 2. and 3. is", sex2_count/sex1_count)

The overall selection rate is  0.113
Selection rate for SEX==1 is 0.0375
Selection rate for SEX==2 is 0.0755
Difference of counts number between 2. and 3. is 228
Ratio of 2. and 3. is 2.013333333333333


There is a new Python package called `fairlearn` that aims to help machine learning researchers use fairness techniques more easily. Go ahead and download this package (`pip install fairlearn` or `conda install fairlearn`). Note: the authors of `fairlearn` call 'statistical parity' from the textbook as 'demographic parity'.

We will use `selection_rate`, `demographic_parity_difference`, `demographic_parity_ratio` from `fairlearn.metrics` to calculate 1., 4. and 5. above. The way to use these functions is: first argument `Y_test`, second argument `Y_prediction`, and for `demographic_parity_difference` and `demographic_parity_ratio` you need to give a keyword argument for the 'sensitive feature' by `sensitive_features=SF_test`.

Use these three functions and confirm your earlier calculations for 1., 4. and 5.

In [6]:
pip install fairlearn

     |████████████████████████████████| 24.6MB 1.5MB/s 


In [7]:
from fairlearn.metrics import selection_rate, demographic_parity_difference, demographic_parity_ratio

print("selection rate is", selection_rate(y_test,y_predict,pos_label=1))
print("demographic parity difference is",demographic_parity_difference(y_test,y_predict,sensitive_features=SF_test))
print("demographic parity ratio is",demographic_parity_ratio(y_test,y_predict,sensitive_features=SF_test))

selection rate is 0.113
demographic parity difference is 0.02746679536300732
demographic parity ratio is 0.7779615792067709


### Part c
### Mitigating Unfairness with Giving Weights to Datapoints

Now we are going to use the `GridSearch` function from `fairlearn.reductions` to try to mitigate this statistical parity discrepancy. This is one of the many ways to achieve this. What this function does is it gives higher weights to certain datapoints (determined by the sensitive feature) in order to change what the model learns. It automatically uses grid search to determine which weight should be given to `SEX==1` datapoints vs `SEX==2` datapoints (if you're interested you can read the source code). While it is called grid search, the space it searches is 1-dimensional. For our purposes, `GridSearch` takes four arguments: 

1. The model you defined above. This is a non-keyword argument.
2. Constraint. This needs to be an object defined by `fairlearn`, the class we will be using is `fairlearn.reductions.DemographicParity`. It corresponds to minimizing the difference in selection rates (4. from above). This is a non-keyword argument.
3. (Keyword argument) `grid_limit`. This determines the boundaries of the grid search. Basically, the `GridSearch` function will search the interval \[-`grid_limit`,`grid_limit`\] to find the weights. Choose this to be 0.8.
3. (Keyword argument) `grid_size`. This determines how many points there will be in the grid search. Choose this to be 21.

Just like `sklearn.model_selection.GridSearchCV` we have seen in HW1, `fairlearn.reductions.GridSearch` is also a wrapper, so you need to call `.fit()` on this wrapper (not the model directly). Training should be done in less than 5 minutes. In the end this wrapper will contain `grid_size` many models, each corresponding to a different weight for datapoints.

In [48]:
from fairlearn.reductions import GridSearch, DemographicParity
fairlearn_gs = GridSearch(estimator=clf,constraints=DemographicParity(),grid_size=21,grid_limit=0.2)
fairlearn_gs.fit(X_train,y_train,sensitive_features=X_train[:,1])
predictors = fairlearn_gs.predictors_

The `GridSearch` wrapper object you trained has a field called `predictors_`. This gives a list of all the models trained in this grid search, which will be of size `grid_size`.

In [49]:
predictions_sweep = [predictor.predict(X_test) for predictor in fairlearn_gs.predictors_] 
accuracy_sweep = [predictor.score(X_test, y_test) for predictor in fairlearn_gs.predictors_] 

Notice that `predictions_sweep` is a 21 element list. Each of these elements is a numpy array that contains the predictions (`Y_pred`) for `X_test` for that particular model. Use `demographic_parity_difference` like you did above to find the difference in selection rates for each of these models and store these 'fairness metrics' in a numpy array (it too will be size 21).

In [50]:
from sklearn.metrics import accuracy_score
fairness_metrics = []
for element in predictions_sweep: 
  fairness_metrics.append(demographic_parity_difference(y_test,element,sensitive_features=SF_test))
fairness_metrics

[0.07680013305681686,
 0.058316308009712034,
 0.05376607322142793,
 0.04882432996221803,
 0.03352536559089046,
 0.024332858820083153,
 0.018844965915870523,
 0.021611921666536624,
 0.010947286981782711,
 0.026956853243657028,
 0.02746679536300732,
 0.029950865219704942,
 0.03532032447549312,
 0.04008103010644079,
 0.06372688032688621,
 0.0696062817020247,
 0.08151972562637852,
 0.08848161522005535,
 0.0931699057996981,
 0.09558156060509082,
 0.09713778341732701]

Plot the accuracy vs difference in selection rates for all these models in a scatter plot.

In [26]:
pip install ipympl

In [20]:
pip install matplotlib

In [51]:
import matplotlib.pyplot as plt
# %matplotlib widget
plt.figure()
plt.scatter(accuracy_sweep,fairness_metrics,label="GridSearch")
plt.xlabel("Accuracy")
plt.ylabel("Demographic Parity Difference")
plt.show();

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Part d 

Now calculate the *pareto front*: go over each (accuracy, difference in selection rates) pair and compare it to every other pair. If you find the first pair to have lower accuracy and higher difference in selection rate, discard it.

In [52]:
accuracy_sweep_pareto = np.copy(accuracy_sweep)
fairness_metrics_pareto = np.copy(fairness_metrics)
pareto = np.column_stack((accuracy_sweep_pareto,fairness_metrics_pareto))
sort_pareto = pareto[np.argsort(pareto[:, 0])]
sort_pareto = sort_pareto[::-1]

max_accuracy = sort_pareto[-1,0]
min_parity = sort_pareto[-1,1]

for i in range(np.size(sort_pareto,0)):
  if sort_pareto[i,1] <= min_parity:
    print(sort_pareto[i,1])
    min_parity = sort_pareto[i,1]
  else:
    index = np.where(pareto[:,1] == sort_pareto[i,1])
    print("delete", index[0])
    pareto = np.delete(pareto,index[0],0)

pareto

0.018844965915870523
delete [7]
delete [5]
delete [4]
delete [3]
delete [2]
delete [1]
delete [13]
delete [13]
delete [12]
delete [9]
delete [10]
delete [0]
delete [8]
delete [7]
0.010947286981782711
delete [4]
delete [2]
delete [2]
delete [2]
delete [2]


array([[0.848     , 0.01884497],
       [0.8395    , 0.01094729]])

* Plot the accuracy vs difference in selection rates for all the models the grid search in a scatter plot.
* On the same plot, plot the accuracy vs difference in selection rates for the points on the pareto front. Make these points red.

In [53]:
plt.figure()
plt.scatter(accuracy_sweep,fairness_metrics,label="GridSearch")
plt.plot(pareto[:,0],pareto[:,1],"o-",label="Pareto",color="tab:red")
plt.xlabel("Accuracy")
plt.ylabel("Demographic Parity Difference")
plt.legend()
plt.show();

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Which one of these models would you choose to use? Why?

## Problem 3

This problem builds on ideas from paper 2a in Module 1: “Against prediction: Sentencing, Policing, And Punishing In An Actuarial Age,” Chicago Public Law And Legal Theory Working Paper No. 94, Harcourt, 2005. 

Suppose we have two groups $A$ and $B$ of population sizes $n$ and $9n$. We find that under equal surveillance, group $A$’s offending rate is $2$ offenses per month per person and that of group $B$ is $1$ offense per person per month (See Figure on page 17 of the above paper). Equal surveillance means that out of a total of $10m$ monitors, $m$ monitors are assigned to group $A$ and $9m$ monitors are assigned to group $B$. The surveillance is $\frac{m}{n}$ per capita for both groups. So, under equal surveillance, the combined offending rate over both groups is $11n$ per month. 

Suppose we want to bring down the overall offending rate by increasing the monitors for group $A$. In order to do that we collected some data and found that we could model the rate of decrease in offenses for group $A$ by a function as follows:
	If we increase the per capita surveillance from $\frac{m}{n}$ to $\frac{m}{n} (1 + \delta_A)$, the offending rate declines to $2e^{- \alpha \delta_A}$. ($\alpha>0$)

Similarly, we modeled the rate of increase in offenses for group $B$ by a function under reduced surveillance as follows:
	If we decrease the per capita monitoring rate from $\frac{m}{n}$ to $\frac{m}{n} (1 - \delta_B)$, the offending rate rises to $e^{ \beta \delta_B}$. ($\beta>0$)

How would you distribute the monitors so that 

**a)** Both groups had the same offending rate?

**b)** The combined offending amount was minimized?


Consider 3 different scenarios when $\alpha > \beta$, $\alpha = \beta$, and $\alpha < \beta$. 

**c)** For each of the scenarios, which of the above solutions would you prefer and why?

**TIPS**:
- For **part a** and **part b**, answer by calculating $\delta_A$ and $\delta_B$.
- Try to set up equations from the given information. This problem mostly consists of algebra.
- If you do it correctly, **part c** should makes sense immediately.


a)

Known variables: $\alpha, \beta, m, n$. ($\alpha, \beta>0$）


Unknown variables: $\delta_B, \delta_A$, and the number of devices shifted, which is x.

The three equations are:

1.    $\frac{m}{n} (1 + \delta_A) = \frac{m+x}{n}$
2.   $\frac{m}{n} (1 - \delta_B) = \frac{9m-x}{9n}$
3.   $2e^{- \alpha \delta_A} = e^{ \beta \delta_B}$

By solving the first two equations, we can get that $\delta_A =9 \delta_B$, substitute into the third equation, we get $2e^{- 9 \alpha \delta_B} = e^{ \beta \delta_B}$. Upon solving, we get $\delta_B = \frac{ln(2)}{9\alpha+\beta}$, $\delta_A = 9\frac{ln(2)}{9\alpha+\beta}$.


b)

Same as a), we have $\delta_A =9 \delta_B$. The third equation changed to $\frac{d}{d\delta_A}(2e^{- \alpha \delta_A} \times n + e^{ \beta \delta_B} \times 9n) = 0$. After substitution, we have $\frac{d}{d\delta_B}(2e^{- 9\alpha \delta_B} \times n + e^{ \beta \delta_B} \times 9n) = 0$.

After solving the equation, we get $\delta_B = \frac{ln(\frac{2\alpha}{\beta})}{9\alpha+\beta}$, $\delta_A = 9\frac{ln(\frac{2\alpha}{\beta})}{9\alpha+\beta}$.

c) 
Plot combined offering rate vs. $\delta_B$ for $\alpha$ and $\beta$ ranging from 0 to 3. Reference: https://ipywidgets.readthedocs.io/en/stable/examples/Using%20Interact.html#Flickering-and-jumping-output

In [62]:
from ipywidgets import interactive
import math

# plot the complete graph
alpha = np.linspace(0.,3.,num=100)
beta = np.linspace(0.,3.,num=100)
delta_b = np.log(2)/(9*alpha+beta)
print(np.shape(delta_b))
plt.plot(delta_b,function(alpha,beta,delta_b))
plt.ylim(1,4)
plt.show()

def function(alpha,beta,delta_b):
  return 2* math.exp(-9*alpha*delta_b) + math.exp(beta*delta_b)

# plot the interactive marker
fig, ax = plt.subplots()
def update(alpha,beta):
  delta_b = np.log(2)/(9*alpha+beta)
  plt.scatter(delta_b,function(alpha,beta,delta_b),c=colr)
  plt.ylim(1,4)
  # fig.canvas.draw_idle()
  # plt.show()
  return()

interactive(update, alpha=(0.,3.,0.01),beta=(0.,3.,0.01),colr=["red","orange"])

(100,)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in true_divide
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in multiply
  import sys


TypeError: ignored